# Tune GPT2 to generate non-toxic tweets


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install transformers trl wandb datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

### Configuration

In [ ]:
config = PPOConfig(
    model_name="redrussianarmy/gpt2-turkish-cased",
    learning_rate=1.41e-5,
    log_with="wandb",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [ ]:
import wandb

wandb.init()

wandb: Currently logged in as: yankihue (data-aug-rlhf). Use `wandb login --relogin` to force relogin


In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset("Overfit-GM/turkish-toxic-language", split="train[:8000]")
    ds = ds.remove_columns("target")
    ds = ds.remove_columns("source")


    def filter_fn(sample):
        is_toxic = sample["is_toxic"]
        return is_toxic != 0 

    ds = ds.filter(filter_fn, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["text"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4087 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1247 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.eos_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
classification_pipe = pipeline("sentiment-analysis", "Overfit-GM/mdeberta-v3-base-offensive", device=device, return_all_scores=True,)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
text = "kardeşim sen manyak mısın"
output = classification_pipe(text)
output

[[{'label': 'INSULT', 'score': 0.9889655113220215},
  {'label': 'OTHER', 'score': 0.00017199355352204293},
  {'label': 'PROFANITY', 'score': 0.010466167703270912},
  {'label': 'RACIST', 'score': 0.00021687541448045522},
  {'label': 'SEXIST', 'score': 0.0001794756535673514}]]

In [ ]:
text = "nasılsın aq"
output = classification_pipe(text)
output

[[{'label': 'INSULT', 'score': 0.6057938933372498},
  {'label': 'OTHER', 'score': 0.00041051520383916795},
  {'label': 'PROFANITY', 'score': 0.3894211947917938},
  {'label': 'RACIST', 'score': 0.0008898780797608197},
  {'label': 'SEXIST', 'score': 0.0034845122136175632}]]

In [ ]:
text = "karşimmmm keyifler nası"
output = classification_pipe(text)
output

[[{'label': 'INSULT', 'score': 4.0885937778512016e-05},
  {'label': 'OTHER', 'score': 0.9999338388442993},
  {'label': 'PROFANITY', 'score': 1.6460691767861135e-05},
  {'label': 'RACIST', 'score': 6.749761723767733e-06},
  {'label': 'SEXIST', 'score': 2.0534121176751796e-06}]]

In [ ]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = classification_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
7it [15:37, 133.82s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
15it [33:40, 134.71s/it]


In [ ]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in classification_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in classification_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,Yahudi,"asıllı soy kardeşi Abdulmuttalibim, Muttal",banka bilimsel konsept evlilik nedeniyle yeni...,-1.701994,4.330927
1,Ben Rick James,Fishdoll hakkında,", istatistiksel ve analitik gelişme",-1.032374,7.978185
2,center,gibi sanatçılarla kulvarlarda,bölümler düzenleme düzenleme konusunda daimi,8.259436,9.173103
3,La siktir git am,"k...\nAşk’ın filistin pornosu gibi,, bir kurgu...","ve bir kadının performansını düzenleyerek, bu...",-3.185980,-2.904784
4,senin de amına,"koyduğumda, 59-65 aralıktan sonra beni burada...",programlarına organize bir şekilde konuşmalar...,-2.852715,4.227712
5,Bi insanın bacaklarına,kondurduksu zincirlerini ve uzun,"bir rahatlama, bir iyileşme süreci,",-1.114196,0.761599
6,Blah Blah B,lah Blah Chalil,", atan etkinlik konusunda geniş çalışmaların",-3.323943,9.084172
7,Ümmet isteyen yal,dızlı! Böyle değil,yıldan kısa bir şekilde derslere,-3.406998,2.196583
8,Allah belanızı versin bok vardı,şimdi burda zamanı olmayan bir program var si...,.\n13. gün boyunca 7.5.000. oyun,-3.525647,-3.561754
9,Tayyip,Erdoğan’ın açıklamalarının kadına şiddeti art...,daha fazla daha verim sağlamak ve daha fazla ...,-0.922751,9.239885


In [ ]:
df_results.to_csv()

',query,response (before),response (after),rewards (before),rewards (after)\n0,Yahudi," asıllı soy kardeşi Abdulmuttalibim, Muttal", banka bilimsel konsept evlilik nedeniyle yeni etkinlik konusunda oldukça önemli bir,-1.7019941806793213,4.330926895141602\n1,Ben Rick James, Fishdoll hakkında,", istatistiksel ve analitik gelişme",-1.0323736667633057,7.978184700012207\n2,center, gibi sanatçılarla kulvarlarda, bölümler düzenleme düzenleme konusunda daimi,8.259435653686523,9.173103332519531\n3,La siktir git am,"k...\nAşk’ın filistin pornosu gibi,, bir kurgu yaparsan"," ve bir kadının performansını düzenleyerek, bu konuda bir şekilde bir öğrenciler, 1",-3.1859800815582275,-2.9047839641571045\n4,senin de amına," koyduğumda, 59-65 aralıktan sonra beni buradan alayım. Al", programlarına organize bir şekilde konuşmalar kazandıran ve bazı konuları izleyen bir şekilde eğitimle özveri,-2.852715253829956,4.2277116775512695\n5,Bi insanın bacaklarına, kondurduksu zincirlerini ve uzun," bir rahatlama, 

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    0.499236
rewards (after)     5.212398
dtype: float64


median:


rewards (before)   -1.408095
rewards (after)     8.487312
dtype: float64

In [ ]:
from huggingface_hub import notebook_login, create_repo
notebook_login()
model.save_pretrained("final-gpt2-tr-detoxified-final", push_to_hub=True)
tokenizer.save_pretrained("final-gpt2-tr-detoxified-final", push_to_hub=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

('gpt2-tr-detoxified-final/tokenizer_config.json',
 'gpt2-tr-detoxified-final/special_tokens_map.json',
 'gpt2-tr-detoxified-final/vocab.json',
 'gpt2-tr-detoxified-final/merges.txt',
 'gpt2-tr-detoxified-final/added_tokens.json',
 'gpt2-tr-detoxified-final/tokenizer.json')